# * VINSIGHT : Inflow
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R00010001CS' --Total Inflow M1
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'TB0R00010001CG' --Total Inflow M1 : TRUE - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 : DTAC - GEO Channel
        
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'TB1R000900CG' --Prepaid Inflow M1 : TMH - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 : DTAC - GEO Channel
        
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C : TMH - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C : DTAC - GEO Channel
        , 'B2R020500CS' --Postpaid Inflow M1 B2B
        , 'TB2R020500CS' --Postpaid Inflow M1 B2B : TMH
        , 'DB2R020500CS' --Postpaid Inflow M1 B2B : DTAC
        , 'B2R020500CG' --Postpaid Inflow M1 B2B - GEO Channel
        , 'DB2R020500CG' --Postpaid Inflow M1 B2B : DTAC - GEO Channel
        , 'TB2R020500CG' --Postpaid Inflow M1 B2B : TMH - GEO Channel
        
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB3R000601D1CS' --TOL Inflow M1 Connected : Consumer (Install Location)
        , 'TB3R000601D1CG' --TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB3R000602CG' --TOL Inflow M1 Connected : DataService - GEO Channel
        , 'TB3R000602D1CS' --TOL Inflow M1 Connected : DataService (Install Location)
        , 'TB3R000602D1CG' --TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

        , 'TB4R001700CS' --TVS Now Inflow M1
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001004CG' --TVS CMDU Inflow M1 - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-07-11, 11:06:12

DataFrame: 6876 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_20600\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Inflow M1(All Channel) Yearly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_yearly_df = m1_yearly_df.loc[m1_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_yearly_df['TOTAL'] = np.where(m1_yearly_df['METRIC_CD']=='B0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOTAL_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB0R00010001CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE'] = np.where(m1_yearly_df['METRIC_CD']=='B1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['PRE_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB1R000900CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C'] = np.where(m1_yearly_df['METRIC_CD']=='B2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R010500CS', m1_yearly_df['P'], 0)
m1_yearly_df['POST_B2C_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R010500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B'] = np.where(m1_yearly_df['METRIC_CD']=='B2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_T'] = np.where(m1_yearly_df['METRIC_CD']=='TB2R020500CS', m1_yearly_df['P'], 0)
# m1_yearly_df['POST_B2B_D'] = np.where(m1_yearly_df['METRIC_CD']=='DB2R020500CS', m1_yearly_df['P'], 0)
m1_yearly_df['TOL_CONN'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_CONN(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000601D1CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TOL_DSV(Install)'] = np.where(m1_yearly_df['METRIC_CD']=='TB3R000602D1CS', m1_yearly_df['P'], 0)
m1_yearly_df['TVS_NOW'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001700CS', m1_yearly_df['P'], 0)
# m1_yearly_df['TVS_CMDU'] = np.where(m1_yearly_df['METRIC_CD']=='TB4R001004CS', m1_yearly_df['P'], 0)

# m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
# m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_yearly_df = m1_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_yearly_df['YoY'] = m1_yearly_df['TOTAL'] - m1_yearly_df['TOTAL'].shift()
m1_yearly_df = m1_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_yearly_df = m1_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_yearly_df_display = m1_yearly_df.copy()
m1_yearly_df_display['ACTUAL_AS_OF'] = m1_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_yearly_df_display[col] = m1_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-07-11 04:27:10,20250709,0,"3,470,577,086","1,834,958,341","1,635,618,745","2,631,749,344","1,212,681,787","1,425,372,613","513,531,637","400,627,660","114,775,558","169,829,711","2,377,880"


In [5]:
''' Inflow M1(All Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_quarterly_df = m1_quarterly_df.loc[m1_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_quarterly_df['TOTAL'] = np.where(m1_quarterly_df['METRIC_CD']=='B0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOTAL_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB0R00010001CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE'] = np.where(m1_quarterly_df['METRIC_CD']=='B1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['PRE_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB1R000900CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R010500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['POST_B2C_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R010500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B'] = np.where(m1_quarterly_df['METRIC_CD']=='B2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_T'] = np.where(m1_quarterly_df['METRIC_CD']=='TB2R020500CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['POST_B2B_D'] = np.where(m1_quarterly_df['METRIC_CD']=='DB2R020500CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TOL_CONN'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_CONN(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000601D1CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TOL_DSV(Install)'] = np.where(m1_quarterly_df['METRIC_CD']=='TB3R000602D1CS', m1_quarterly_df['P'], 0)
m1_quarterly_df['TVS_NOW'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001700CS', m1_quarterly_df['P'], 0)
# m1_quarterly_df['TVS_CMDU'] = np.where(m1_quarterly_df['METRIC_CD']=='TB4R001004CS', m1_quarterly_df['P'], 0)

# m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
# m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_quarterly_df = m1_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_quarterly_df['QoQ'] = m1_quarterly_df['TOTAL'] - m1_quarterly_df['TOTAL'].shift()
m1_quarterly_df = m1_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_quarterly_df = m1_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_quarterly_df_display = m1_quarterly_df.copy()
m1_quarterly_df_display['ACTUAL_AS_OF'] = m1_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_quarterly_df_display[col] = m1_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-07-11 04:27:10,20250331,0,"1,778,578,561","918,417,080","860,161,482","1,379,322,046","619,695,416","759,626,629","249,140,023","195,429,182","53,710,841","79,484,928","1,120,189"
1,20252,2025-07-11 04:27:10,20250630,"-219,755,359","1,558,823,203","847,859,509","710,963,693","1,168,938,044","555,464,378","613,473,665","241,679,456","185,021,052","56,658,404","81,798,101","1,174,157"
2,20253,2025-07-11 04:27:10,20250709,"-1,425,647,881","133,175,321","68,681,752","64,493,570","83,489,254","37,521,992","52,272,319","22,712,158","20,177,426","4,406,313","8,546,682","83,534"


In [6]:
''' Inflow M1(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_monthly_df = m1_monthly_df.loc[m1_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_monthly_df['TOTAL'] = np.where(m1_monthly_df['METRIC_CD']=='B0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOTAL_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB0R00010001CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE'] = np.where(m1_monthly_df['METRIC_CD']=='B1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['PRE_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB1R000900CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C'] = np.where(m1_monthly_df['METRIC_CD']=='B2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R010500CS', m1_monthly_df['P'], 0)
m1_monthly_df['POST_B2C_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R010500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B'] = np.where(m1_monthly_df['METRIC_CD']=='B2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_T'] = np.where(m1_monthly_df['METRIC_CD']=='TB2R020500CS', m1_monthly_df['P'], 0)
# m1_monthly_df['POST_B2B_D'] = np.where(m1_monthly_df['METRIC_CD']=='DB2R020500CS', m1_monthly_df['P'], 0)
m1_monthly_df['TOL_CONN'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_CONN(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000601D1CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TOL_DSV(Install)'] = np.where(m1_monthly_df['METRIC_CD']=='TB3R000602D1CS', m1_monthly_df['P'], 0)
m1_monthly_df['TVS_NOW'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001700CS', m1_monthly_df['P'], 0)
# m1_monthly_df['TVS_CMDU'] = np.where(m1_monthly_df['METRIC_CD']=='TB4R001004CS', m1_monthly_df['P'], 0)

# m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
# m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_monthly_df = m1_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_monthly_df['MoM'] = m1_monthly_df['TOTAL'] - m1_monthly_df['TOTAL'].shift()
m1_monthly_df = m1_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_monthly_df = m1_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_monthly_df_display = m1_monthly_df.copy()
m1_monthly_df_display['ACTUAL_AS_OF'] = m1_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_monthly_df_display[col] = m1_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-07-11 04:27:10,20250131,0,"639,880,599","319,922,636","319,957,964","508,715,782","225,749,189","282,966,592","79,460,514","61,334,089","18,126,426","25,646,982","382,335"
1,202502,2025-07-11 04:27:10,20250228,"-64,700,198","575,180,401","288,071,249","287,109,152","442,103,679","192,368,192","249,735,487","81,132,607","63,185,463","17,947,144","25,046,745","267,183"
2,202503,2025-07-11 04:27:10,20250331,"-11,662,840","563,517,561","310,423,195","253,094,366","428,502,585","201,578,035","226,924,550","88,546,902","70,909,630","17,637,272","28,791,201","470,671"
3,202504,2025-07-11 04:27:10,20250430,"-29,481,414","534,036,147","295,795,804","238,240,343","395,842,378","191,822,637","204,019,741","87,789,592","69,118,915","18,670,677","26,236,110","441,580"
4,202505,2025-07-11 04:27:10,20250531,"-1,532,940","532,503,207","287,111,076","245,392,131","400,164,161","190,514,414","209,649,746","79,167,109","59,723,743","19,443,366","28,292,314","401,782"
5,202506,2025-07-11 04:27:10,20250630,"-40,219,358","492,283,849","264,952,630","227,331,219","372,931,505","173,127,327","199,804,178","74,722,754","56,178,394","18,544,361","27,269,677","330,796"
6,202507,2025-07-11 04:27:10,20250709,"-359,108,528","133,175,321","68,681,752","64,493,570","83,489,254","37,521,992","52,272,319","22,712,158","20,177,426","4,406,313","8,546,682","83,534"


In [7]:
''' Inflow M1(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_weekly_df = m1_weekly_df.loc[m1_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_weekly_df['TOTAL'] = np.where(m1_weekly_df['METRIC_CD']=='B0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOTAL_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB0R00010001CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE'] = np.where(m1_weekly_df['METRIC_CD']=='B1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['PRE_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB1R000900CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C'] = np.where(m1_weekly_df['METRIC_CD']=='B2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R010500CS', m1_weekly_df['P'], 0)
m1_weekly_df['POST_B2C_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R010500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B'] = np.where(m1_weekly_df['METRIC_CD']=='B2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_T'] = np.where(m1_weekly_df['METRIC_CD']=='TB2R020500CS', m1_weekly_df['P'], 0)
# m1_weekly_df['POST_B2B_D'] = np.where(m1_weekly_df['METRIC_CD']=='DB2R020500CS', m1_weekly_df['P'], 0)
m1_weekly_df['TOL_CONN'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_CONN(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000601D1CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TOL_DSV(Install)'] = np.where(m1_weekly_df['METRIC_CD']=='TB3R000602D1CS', m1_weekly_df['P'], 0)
m1_weekly_df['TVS_NOW'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001700CS', m1_weekly_df['P'], 0)
# m1_weekly_df['TVS_CMDU'] = np.where(m1_weekly_df['METRIC_CD']=='TB4R001004CS', m1_weekly_df['P'], 0)

# m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
# m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_weekly_df = m1_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_weekly_df['WoW'] = m1_weekly_df['TOTAL'] - m1_weekly_df['TOTAL'].shift()
m1_weekly_df = m1_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_weekly_df = m1_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_weekly_df_display = m1_weekly_df.copy()
m1_weekly_df_display['ACTUAL_AS_OF'] = m1_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_weekly_df_display[col] = m1_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025022,2025-07-11 04:27:10,20250601,0,"15,959,382","8,363,460","7,595,922","12,057,691","5,043,697","7,013,994","2,837,264","2,316,531","520,733","886,720","13,403"
1,2025023,2025-07-11 04:27:10,20250608,"92,930,203","108,889,585","58,125,733","50,763,851","80,639,520","35,669,535","44,969,985","18,716,633","14,427,733","4,288,900","6,636,901","81,933"
2,2025024,2025-07-11 04:27:10,20250615,"-2,299,705","106,589,880","59,610,936","46,978,943","80,227,694","38,816,433","41,411,261","17,138,181","13,171,528","3,966,653","6,221,571","57,082"
3,2025025,2025-07-11 04:27:10,20250622,"7,376,073","113,965,953","60,009,397","53,956,556","84,519,158","39,528,333","44,990,825","16,739,014","11,847,431","4,891,583","6,374,313","77,002"
4,2025026,2025-07-11 04:27:10,20250629,"9,629,624","123,595,577","65,976,956","57,618,621","97,230,709","45,394,131","51,836,578","16,469,618","12,339,214","4,130,404","6,113,219","78,754"
5,2025027,2025-07-11 04:27:10,20250706,"3,021,415","126,616,992","66,083,604","60,533,388","90,072,010","40,834,843","49,237,167","19,865,593","15,956,007","3,909,586","6,660,724","86,513"
6,2025028,2025-07-11 04:27:10,20250709,"-96,775,189","29,841,803","15,464,294","14,377,508","11,673,977","5,362,347","12,616,687","5,668,608","6,297,374","1,242,815","2,922,911","19,643"


In [8]:
''' Inflow M1(All Channel) Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1
    , 'TB0R00010001CS' #Total Inflow M1 : TRUE
    , 'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'B1R000900CS' #Prepaid Inflow M1
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'B2R010500CS' #Postpaid Inflow M1 B2C
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    # , 'B2R020500CS' #Postpaid Inflow M1 B2B
    # , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    # , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    # , 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
    # , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    # , 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

    , 'TB4R001700CS' #TVS Now Inflow M1
    # , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

m1_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_daily_df = m1_daily_df.loc[m1_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_daily_df['TOTAL'] = np.where(m1_daily_df['METRIC_CD']=='B0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_T'] = np.where(m1_daily_df['METRIC_CD']=='TB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['TOTAL_D'] = np.where(m1_daily_df['METRIC_CD']=='DB0R00010001CS', m1_daily_df['P'], 0)
m1_daily_df['PRE'] = np.where(m1_daily_df['METRIC_CD']=='B1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_T'] = np.where(m1_daily_df['METRIC_CD']=='TB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['PRE_D'] = np.where(m1_daily_df['METRIC_CD']=='DB1R000900CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C'] = np.where(m1_daily_df['METRIC_CD']=='B2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R010500CS', m1_daily_df['P'], 0)
m1_daily_df['POST_B2C_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R010500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B'] = np.where(m1_daily_df['METRIC_CD']=='B2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_T'] = np.where(m1_daily_df['METRIC_CD']=='TB2R020500CS', m1_daily_df['P'], 0)
# m1_daily_df['POST_B2B_D'] = np.where(m1_daily_df['METRIC_CD']=='DB2R020500CS', m1_daily_df['P'], 0)
m1_daily_df['TOL_CONN'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_CONN(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000601D1CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602CS', m1_daily_df['P'], 0)
# m1_daily_df['TOL_DSV(Install)'] = np.where(m1_daily_df['METRIC_CD']=='TB3R000602D1CS', m1_daily_df['P'], 0)
m1_daily_df['TVS_NOW'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001700CS', m1_daily_df['P'], 0)
# m1_daily_df['TVS_CMDU'] = np.where(m1_daily_df['METRIC_CD']=='TB4R001004CS', m1_daily_df['P'], 0)

# m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
# m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_daily_df = m1_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_daily_df['DoD'] = m1_daily_df['TOTAL'] - m1_daily_df['TOTAL'].shift()
m1_daily_df = m1_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_daily_df = m1_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_daily_df_display = m1_daily_df.copy()
mod_col_list = m1_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_daily_df_display[col] = m1_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202507,20250710,2025-07-11 04:27:10,"-3,072,399",0,0,0,0,0,0,0,0,0,0,0
1,202507,20250709,2025-07-11 04:27:10,"-7,764,011","3,072,399","3,072,399",0,0,0,0,0,"1,871,581",0,"955,481",0
2,202507,20250708,2025-07-11 04:27:10,"-5,096,583","10,836,410","3,456,812","7,379,599",0,0,"6,305,057","2,851,693","2,218,997","632,696","942,910","6,967"
3,202507,20250707,2025-07-11 04:27:10,"-1,164,258","15,932,993","8,935,084","6,997,910","11,673,977","5,362,347","6,311,631","2,816,915","2,206,796","610,119","1,024,520","12,676"
4,202507,20250706,2025-07-11 04:27:10,"360,651","17,097,251","9,114,137","7,983,114","12,779,732","5,311,947","7,467,785","3,370,776","2,882,529","488,247","900,025","15,531"
5,202507,20250705,2025-07-11 04:27:10,"968,930","16,736,600","9,014,916","7,721,684","12,580,766","5,385,042","7,195,724","3,077,717","2,571,951","505,766","976,118","10,353"
6,202507,20250704,2025-07-11 04:27:10,"47,226","15,767,670","8,764,306","7,003,364","11,881,394","5,384,141","6,497,252","2,576,246","2,177,572","398,674","933,941","7,580"
7,202507,20250703,2025-07-11 04:27:10,"-1,152,222","15,720,444","8,703,836","7,016,607","11,884,410","5,497,121","6,387,288","2,521,441","2,009,958","511,483","942,420","5,802"
8,202507,20250702,2025-07-11 04:27:10,"-4,266,222","16,872,666","8,657,208","8,215,458","11,480,426","5,227,048","6,253,378","2,628,259","2,062,769","565,490","931,636","12,284"
9,202507,20250701,2025-07-11 04:27:10,"-2,144,585","21,138,888","8,963,055","12,175,833","11,208,550","5,354,347","5,854,203","2,869,110","2,175,272","693,838","939,631","12,341"


## Overview(Geo) by Period

In [9]:
''' Inflow M1(Geo Channel) Yearly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_yearly_df = m1_geo_yearly_df.loc[m1_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_yearly_df['TOTAL'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOTAL_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['PRE_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB1R000900CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R010500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['POST_B2C_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R010500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B'] = np.where(m1_geo_yearly_df['METRIC_CD']=='B2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_T'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB2R020500CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['POST_B2B_D'] = np.where(m1_geo_yearly_df['METRIC_CD']=='DB2R020500CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TOL_CONN'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_CONN(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TOL_DSV(Install)'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_yearly_df['P'], 0)
m1_geo_yearly_df['TVS_NOW'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001700CG', m1_geo_yearly_df['P'], 0)
# m1_geo_yearly_df['TVS_CMDU'] = np.where(m1_geo_yearly_df['METRIC_CD']=='TB4R001004CG', m1_geo_yearly_df['P'], 0)

# m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
# m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_yearly_df = m1_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_yearly_df['YoY'] = m1_geo_yearly_df['TOTAL'] - m1_geo_yearly_df['TOTAL'].shift()
m1_geo_yearly_df = m1_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
m1_geo_yearly_df = m1_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_yearly_df_display = m1_geo_yearly_df.copy()
m1_geo_yearly_df_display['ACTUAL_AS_OF'] = m1_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_yearly_df_display[col] = m1_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025,2025-07-11 04:27:10,20250709,0,"2,734,581,040","1,562,513,810","1,172,067,230","2,318,847,905","1,200,765,325","1,123,191,550","250,872,207","205,475,733","46,307,418","133,957,938","2,367,584"


In [10]:
''' Inflow M1(Geo Channel) Quarterly '''

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
m1_geo_quarterly_df = m1_geo_quarterly_df.loc[m1_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_quarterly_df['TOTAL'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOTAL_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['PRE_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB1R000900CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R010500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['POST_B2C_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R010500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='B2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_T'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB2R020500CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['POST_B2B_D'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='DB2R020500CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TOL_CONN'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_CONN(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TOL_DSV(Install)'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_quarterly_df['P'], 0)
m1_geo_quarterly_df['TVS_NOW'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001700CG', m1_geo_quarterly_df['P'], 0)
# m1_geo_quarterly_df['TVS_CMDU'] = np.where(m1_geo_quarterly_df['METRIC_CD']=='TB4R001004CG', m1_geo_quarterly_df['P'], 0)

# m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
# m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_quarterly_df = m1_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_quarterly_df['QoQ'] = m1_geo_quarterly_df['TOTAL'] - m1_geo_quarterly_df['TOTAL'].shift()
m1_geo_quarterly_df = m1_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
m1_geo_quarterly_df = m1_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_quarterly_df_display = m1_geo_quarterly_df.copy()
m1_geo_quarterly_df_display['ACTUAL_AS_OF'] = m1_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_quarterly_df_display[col] = m1_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,20251,2025-07-11 04:27:10,20250331,0,"1,378,838,395","790,266,697","588,571,699","1,179,496,714","614,011,326","565,485,388","124,890,149","102,959,079","21,931,070","62,397,203","1,111,862"
1,20252,2025-07-11 04:27:10,20250630,"-122,475,864","1,256,362,531","717,936,814","538,425,718","1,064,571,804","549,580,091","514,991,713","115,279,600","93,119,044","22,160,557","64,775,717","1,172,189"
2,20253,2025-07-11 04:27:10,20250709,"-1,156,982,417","99,380,114","54,310,300","45,069,814","74,779,387","37,173,908","42,714,449","10,702,457","9,397,611","2,215,791","6,785,018","83,534"


In [11]:
''' Inflow M1(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
m1_geo_monthly_df = m1_geo_monthly_df.loc[m1_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_monthly_df['TOTAL'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOTAL_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['PRE_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB1R000900CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R010500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['POST_B2C_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R010500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B'] = np.where(m1_geo_monthly_df['METRIC_CD']=='B2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_T'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB2R020500CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['POST_B2B_D'] = np.where(m1_geo_monthly_df['METRIC_CD']=='DB2R020500CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TOL_CONN'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_CONN(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TOL_DSV(Install)'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_monthly_df['P'], 0)
m1_geo_monthly_df['TVS_NOW'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001700CG', m1_geo_monthly_df['P'], 0)
# m1_geo_monthly_df['TVS_CMDU'] = np.where(m1_geo_monthly_df['METRIC_CD']=='TB4R001004CG', m1_geo_monthly_df['P'], 0)

# m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
# m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_monthly_df = m1_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_monthly_df['MoM'] = m1_geo_monthly_df['TOTAL'] - m1_geo_monthly_df['TOTAL'].shift()
m1_geo_monthly_df = m1_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
m1_geo_monthly_df = m1_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_monthly_df_display = m1_geo_monthly_df.copy()
m1_geo_monthly_df_display['ACTUAL_AS_OF'] = m1_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_monthly_df_display[col] = m1_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202501,2025-07-11 04:27:10,20250131,0,"491,850,599","278,217,598","213,633,001","429,927,804","223,753,635","206,174,169","38,555,159","31,477,331","7,077,828","19,471,134","376,664"
1,202502,2025-07-11 04:27:10,20250228,"-52,938,633","438,911,966","247,017,359","191,894,607","375,063,654","190,565,264","184,498,390","39,986,117","32,971,912","7,014,205","19,961,102","264,749"
2,202503,2025-07-11 04:27:10,20250331,"9,163,864","448,075,830","265,031,740","183,044,091","374,505,256","199,692,427","174,812,829","46,348,874","38,509,836","7,839,038","22,964,967","470,449"
3,202504,2025-07-11 04:27:10,20250430,"-18,662,210","429,413,620","251,562,458","177,851,162","359,407,356","189,643,908","169,763,448","45,183,640","37,483,652","7,699,989","20,901,734","441,246"
4,202505,2025-07-11 04:27:10,20250531,"1,471,543","430,885,163","244,187,397","186,697,766","367,391,197","188,609,826","178,781,371","36,970,047","29,453,492","7,516,555","22,391,575","400,240"
5,202506,2025-07-11 04:27:10,20250630,"-34,821,414","396,063,748","222,186,959","173,876,790","337,773,250","171,326,357","166,446,894","33,125,912","26,181,899","6,944,013","21,482,408","330,703"
6,202507,2025-07-11 04:27:10,20250709,"-296,683,635","99,380,114","54,310,300","45,069,814","74,779,387","37,173,908","42,714,449","10,702,457","9,397,611","2,215,791","6,785,018","83,534"


In [12]:
''' Inflow M1(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_weekly_df = m1_geo_weekly_df.loc[m1_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_weekly_df['TOTAL'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOTAL_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB0R00010001CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['PRE_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB1R000900CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R010500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['POST_B2C_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R010500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B'] = np.where(m1_geo_weekly_df['METRIC_CD']=='B2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_T'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB2R020500CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['POST_B2B_D'] = np.where(m1_geo_weekly_df['METRIC_CD']=='DB2R020500CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TOL_CONN'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_CONN(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TOL_DSV(Install)'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_weekly_df['P'], 0)
m1_geo_weekly_df['TVS_NOW'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001700CG', m1_geo_weekly_df['P'], 0)
# m1_geo_weekly_df['TVS_CMDU'] = np.where(m1_geo_weekly_df['METRIC_CD']=='TB4R001004CG', m1_geo_weekly_df['P'], 0)

# m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
# m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_weekly_df = m1_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_weekly_df['WoW'] = m1_geo_weekly_df['TOTAL'] - m1_geo_weekly_df['TOTAL'].shift()
m1_geo_weekly_df = m1_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
m1_geo_weekly_df = m1_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_weekly_df_display = m1_geo_weekly_df.copy()
m1_geo_weekly_df_display['ACTUAL_AS_OF'] = m1_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = m1_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_weekly_df_display[col] = m1_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,2025022,2025-07-11 04:27:10,20250601,0,"13,182,145","6,721,834","6,460,311","11,033,545","4,992,599","6,040,946","1,368,619","980,649","387,970","677,816","13,403"
1,2025023,2025-07-11 04:27:10,20250608,"74,483,973","87,666,118","47,571,266","40,094,852","73,444,118","35,316,994","38,127,123","8,291,958","6,466,199","1,825,759","5,239,826","81,933"
2,2025024,2025-07-11 04:27:10,20250615,"-1,784,492","85,881,626","49,906,777","35,974,849","72,600,856","38,197,545","34,403,311","7,820,040","6,326,604","1,493,436","4,918,030","57,082"
3,2025025,2025-07-11 04:27:10,20250622,"3,539,510","89,421,136","50,863,382","38,557,754","76,152,202","39,138,766","37,013,436","7,341,405","5,889,549","1,451,856","5,034,795","77,002"
4,2025026,2025-07-11 04:27:10,20250629,"10,839,660","100,260,796","56,096,132","44,164,664","87,569,015","45,065,854","42,503,160","7,116,998","5,580,292","1,536,706","4,778,870","78,754"
5,2025027,2025-07-11 04:27:10,20250706,"-3,252,530","97,008,266","54,205,165","42,803,102","81,227,995","40,473,876","40,754,119","9,140,511","7,227,282","1,913,229","5,289,978","86,420"
6,2025028,2025-07-11 04:27:10,20250709,"-74,984,491","22,023,775","11,132,704","10,891,071","10,524,907","5,314,630","10,319,247","2,748,839","3,108,935","550,848","2,328,111","19,643"


In [13]:
''' Inflow M1(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    , 'TB0R00010001CG' #Total Inflow M1 : TRUE - GEO Channel
    , 'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel

    , 'B1R000900CG' #Prepaid Inflow M1 - GEO Channel
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'B2R010500CG' #Postpaid Inflow M1 B2C - GEO Channel
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    # , 'B2R020500CG' #Postpaid Inflow M1 B2B - GEO Channel
    # , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    # , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    # , 'TB3R000601D1CG' #TOL Inflow M1 Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    # , 'TB3R000602D1CG' #TOL Inflow M1 Connected : DataService - GEO Channel (Install Location)

    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    # , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

m1_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
m1_geo_daily_df = m1_geo_daily_df.loc[m1_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

m1_geo_daily_df['TOTAL'] = np.where(m1_geo_daily_df['METRIC_CD']=='B0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOTAL_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB0R00010001CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE'] = np.where(m1_geo_daily_df['METRIC_CD']=='B1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['PRE_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB1R000900CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R010500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['POST_B2C_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R010500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B'] = np.where(m1_geo_daily_df['METRIC_CD']=='B2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_T'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB2R020500CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['POST_B2B_D'] = np.where(m1_geo_daily_df['METRIC_CD']=='DB2R020500CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TOL_CONN'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_CONN(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000601D1CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TOL_DSV(Install)'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB3R000602D1CG', m1_geo_daily_df['P'], 0)
m1_geo_daily_df['TVS_NOW'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001700CG', m1_geo_daily_df['P'], 0)
# m1_geo_daily_df['TVS_CMDU'] = np.where(m1_geo_daily_df['METRIC_CD']=='TB4R001004CG', m1_geo_daily_df['P'], 0)

# m1_geo_daily_df = m1_geo_daily_df.groupby('TM_KEY_MTH', 'TM_KEY_DAY').agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
# m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

m1_geo_daily_df = m1_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'TOTAL_T':'sum', 'TOTAL_D':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
m1_geo_daily_df['DoD'] = m1_geo_daily_df['TOTAL'] - m1_geo_daily_df['TOTAL'].shift()
m1_geo_daily_df = m1_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
m1_geo_daily_df = m1_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL', 'TOTAL_T', 'TOTAL_D', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_CONN', 'TVS_NOW']]

m1_geo_daily_df_display = m1_geo_daily_df.copy()
mod_col_list = m1_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    m1_geo_daily_df_display[col] = m1_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
m1_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL,TOTAL_T,TOTAL_D,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_CONN,TVS_NOW
0,202507,20250710,2025-07-11 04:27:10,"-1,748,141",0,0,0,0,0,0,0,0,0,0,0
1,202507,20250709,2025-07-11 04:27:10,"-5,727,383","1,748,141","1,748,141",0,0,0,0,0,"910,944",0,"747,429",0
2,202507,20250708,2025-07-11 04:27:10,"-5,324,586","7,475,524","2,075,748","5,399,776",0,0,"5,108,970","1,471,008","1,188,247","282,761","764,462","6,967"
3,202507,20250707,2025-07-11 04:27:10,"-983,519","12,800,110","7,308,815","5,491,295","10,524,907","5,314,630","5,210,277","1,277,831","1,009,744","268,087","816,220","12,676"
4,202507,20250706,2025-07-11 04:27:10,"378,236","13,783,629","7,079,762","6,703,866","11,562,119","5,258,588","6,303,531","1,503,979","1,130,726","373,253","670,914","15,531"
5,202507,20250705,2025-07-11 04:27:10,"725,820","13,405,392","7,209,388","6,196,005","11,208,315","5,323,437","5,884,878","1,402,076","1,110,012","292,064","729,124","10,353"
6,202507,20250704,2025-07-11 04:27:10,"13,829","12,679,572","7,225,254","5,454,318","10,545,657","5,333,770","5,211,887","1,219,805","986,793","233,012","755,153","7,580"
7,202507,20250703,2025-07-11 04:27:10,"265,074","12,665,743","7,267,882","5,397,861","10,589,623","5,449,156","5,140,467","1,192,064","966,476","225,588","751,002","5,802"
8,202507,20250702,2025-07-11 04:27:10,"-20,662","12,400,670","7,015,118","5,385,552","10,304,715","5,187,595","5,117,120","1,217,936","955,820","262,116","776,214","12,284"
9,202507,20250701,2025-07-11 04:27:10,0,"12,421,332","7,380,191","5,041,141","10,044,050","5,306,731","4,737,319","1,417,759","1,138,849","278,910","774,500","12,341"


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202507


### All Service

In [16]:
''' All Service (All Channel) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_NAME'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,All Services,B0R00010001CS,Total Inflow M1,2025-07-11 04:27:10,"133,175,321","133,175,321","105,729,635","105,733,089","103,979,171"
1,202507,All Services,DB0R00010001CS,Total Inflow M1 : DTAC,2025-07-11 04:27:10,"64,493,570","64,493,570","56,178,236","56,178,236","55,143,370"
2,202507,All Services,TB0R00010001CS,Total Inflow M1 : TRUE,2025-07-11 04:27:10,"68,681,752","68,681,752","49,551,399","49,554,853","48,835,801"


In [17]:
''' All Service (Geo Channel) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_NAME'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,All Services,B0R00010001CG,Total Inflow M1 - GEO Channel,2025-07-11 04:27:10,0,"99,380,114","88,908,009","88,911,811","87,230,554"
1,202507,All Services,DB0R00010001CG,Total Inflow M1 : DTAC - GEO Channel,2025-07-11 04:27:10,0,"45,069,814","44,741,558","44,741,907","43,779,701"
2,202507,All Services,TB0R00010001CG,Total Inflow M1 : TRUE - GEO Channel,2025-07-11 04:27:10,0,"54,310,300","44,166,451","44,169,905","43,450,853"


### Prepaid

In [18]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Prepaid,B1R000900CS,Prepaid Inflow M1,2025-07-11 04:27:10,"83,489,254","83,489,254","72,479,354","72,479,354","70,947,766"
1,202507,Prepaid,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-07-11 04:27:10,"52,272,319","52,272,319","51,466,196","51,466,196","50,431,330"
2,202507,Prepaid,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-07-11 04:27:10,"37,521,992","37,521,992","27,032,483","27,032,483","26,386,629"


In [19]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Prepaid,B1R000900CG,Prepaid Inflow M1 - GEO Channel,2025-07-11 04:27:10,0,"74,779,387","64,385,546","64,385,546","62,917,868"
1,202507,Prepaid,DB1R000900CG,Prepaid Inflow M1 : DTAC - GEO Channel,2025-07-11 04:27:10,0,"42,714,449","42,392,232","42,392,232","41,430,026"
2,202507,Prepaid,TB1R000900CG,Prepaid Inflow M1 : TMH - GEO Channel,2025-07-11 04:27:10,0,"37,173,908","27,031,656","27,031,656","26,385,802"


### Postpaid

In [20]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2R010500CS,Postpaid Inflow M1 B2C,2025-07-11 04:27:10,"22,712,158","22,712,158","17,672,031","17,672,031","17,659,466"
1,202507,Postpaid,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-07-11 04:27:10,"4,406,313","4,406,313","4,367,260","4,367,260","4,367,260"
2,202507,Postpaid,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-07-11 04:27:10,"20,177,426","20,177,426","14,723,194","14,723,194","14,723,194"


In [21]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2R010500CG,Postpaid Inflow M1 B2C - GEO Channel,2025-07-11 04:27:10,0,"10,702,457","10,695,420","10,474,370","10,177,119"
1,202507,Postpaid,DB2R010500CG,Postpaid Inflow M1 B2C : DTAC - GEO Channel,2025-07-11 04:27:10,0,"2,215,791","2,209,752","2,209,752","2,209,752"
2,202507,Postpaid,TB2R010500CG,Postpaid Inflow M1 B2C : TMH - GEO Channel,2025-07-11 04:27:10,0,"9,397,611","9,396,613","9,396,613","9,396,613"


In [22]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2R020500CS,Postpaid Inflow M1 B2B,2025-07-11 04:27:10,"9,614,006","9,614,006","1,077,258","840,701","825,584"
1,202507,Postpaid,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-07-11 04:27:10,"7,814,938","7,814,938","344,780","344,780","344,780"
2,202507,Postpaid,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-07-11 04:27:10,"2,012,519","2,012,519","821,467","822,814","822,814"


In [23]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2R020500CG,Postpaid Inflow M1 B2B - GEO Channel,2025-07-11 04:27:10,0,"886,267","790,896","490,775","542,829"
1,202507,Postpaid,DB2R020500CG,Postpaid Inflow M1 B2B : DTAC - GEO Channel,2025-07-11 04:27:10,0,"139,574","139,574","139,923","139,923"
2,202507,Postpaid,TB2R020500CG,Postpaid Inflow M1 B2B : TMH - GEO Channel,2025-07-11 04:27:10,0,"821,467","821,467","822,814","822,814"


### TOL

In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-07-11 04:27:10,"8,546,682","8,546,682","6,784,950","6,784,950","6,711,752"
1,202507,TOL,TB3R000601D1CS,TOL Inflow M1 Connected : Consumer (Install Lo...,2025-07-11 04:27:10,"8,546,682","8,546,682","8,546,083","8,546,083","8,546,083"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3R000601CG,TOL Inflow M1 Connected : Consumer - GEO Channel,2025-07-11 04:27:10,0,"6,785,018","6,784,419","6,784,419","6,711,221"
1,202507,TOL,TB3R000601D1CG,TOL Inflow M1 Connected : Consumer - GEO Chann...,2025-07-11 04:27:10,0,"6,785,018","6,785,018","6,785,018","6,785,018"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-07-11 04:27:10,"276,608","276,608","42,781","42,781","42,781"
1,202507,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-07-11 04:27:10,"276,608","276,608","42,781","42,781","42,781"


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-07-11 04:27:10,"276,608","276,608","42,781","42,781","42,781"
1,202507,TOL,TB3R000602D1CS,TOL Inflow M1 Connected : DataService (Install...,2025-07-11 04:27:10,"276,608","276,608","42,781","42,781","42,781"


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TVS,TB4R001700CS,TVS Now Inflow M1,2025-07-11 04:27:10,"83,534","83,534","83,534","85,640","85,640"
1,202507,TVS,TB4R001700CG,TVS Now Inflow M1 - GEO Channel,2025-07-11 04:27:10,0,"83,534","83,534","85,640","85,640"


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TVS,TB4R001004CS,TVS CMDU Inflow M1,2025-07-11 04:27:10,"62,991","62,991","62,991","62,991","62,991"
1,202507,TVS,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,2025-07-11 04:27:10,0,"5,981","5,981","5,981","5,981"


## ** Special Metric (All & Geo)

### Inflow (All Channel)
    B0R00010001CS   Total Inflow M1
    TB0R00010001CS  Total Inflow M1 : TRUE
    DB0R00010001CS  Total Inflow M1 : DTAC

In [30]:
''' B0R00010001CS : Total Inflow M1 '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'DB2R010500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'B0R00010001CS' #Total Inflow M1

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_df = total_m1_df.loc[total_m1_df['METRIC_CD'].isin(v_metric_list)]

total_m1_df['TOTAL'] = np.where(total_m1_df['METRIC_CD']=='B0R00010001CS', total_m1_df['P'], 0)

total_m1_df['PRE_T'] = np.where(total_m1_df['METRIC_CD']=='TB1R000900CS', total_m1_df['P'], 0)
total_m1_df['PRE_D'] = np.where(total_m1_df['METRIC_CD']=='DB1R000900CS', total_m1_df['P'], 0)

total_m1_df['POST_B2C_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2C_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R010500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_T'] = np.where(total_m1_df['METRIC_CD']=='TB2R020500CS', total_m1_df['P'], 0)
total_m1_df['POST_B2B_D'] = np.where(total_m1_df['METRIC_CD']=='DB2R020500CS', total_m1_df['P'], 0)

total_m1_df['TOL_CON'] = np.where(total_m1_df['METRIC_CD']=='TB3R000601CS', total_m1_df['P'], 0)
total_m1_df['TOL_DSV'] = np.where(total_m1_df['METRIC_CD']=='TB3R000602CS', total_m1_df['P'], 0)

total_m1_df['TVS_NOW'] = np.where(total_m1_df['METRIC_CD']=='TB4R001700CS', total_m1_df['P'], 0)
total_m1_df['TVS_CMDU'] = np.where(total_m1_df['METRIC_CD']=='TB4R001004CS', total_m1_df['P'], 0)

total_m1_df['CHK_SUM'] = np.where(total_m1_df['METRIC_CD'].isin(['TB1R000900CS', 'DB1R000900CS', 'TB2R010500CS', 'DB2R010500CS', 'TB2R020500CS', 'DB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_df['P'], 0)

total_m1_df = total_m1_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_df['CHK_DIFF'] = total_m1_df['TOTAL'] - total_m1_df['CHK_SUM']
total_m1_df = total_m1_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_df = total_m1_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_df[col] = total_m1_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-07-11 04:27:10,0,"639,880,599","639,880,599","225,749,189","282,966,592","61,334,089","18,126,426","4,946,013","18,864,945","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-07-11 04:27:10,0,"575,180,401","575,180,401","192,368,192","249,735,487","63,185,463","17,947,144","4,637,783","19,426,521","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-07-11 04:27:10,0,"563,517,561","563,517,561","201,578,035","226,924,550","70,909,630","17,637,272","5,801,792","8,532,545","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-07-11 04:27:10,-0,"534,036,147","534,036,147","191,822,637","204,019,741","69,118,915","18,670,677","5,402,830","15,549,925","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-07-11 04:27:10,0,"532,503,207","532,503,207","190,514,414","209,649,746","59,723,743","19,443,366","6,239,659","16,299,019","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-07-11 04:27:10,0,"492,283,849","492,283,849","173,127,327","199,804,178","56,178,394","18,544,361","6,226,938","8,982,680","27,269,677","1,643,369","330,796","176,129"
6,202507,2025-07-11 04:27:10,0,"133,175,321","133,175,321","37,521,992","52,272,319","20,177,426","4,406,313","2,012,519","7,814,938","8,546,682","276,608","83,534","62,991"


In [31]:
''' TB0R00010001CS : Total Inflow M1 : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS')

v_metric_list = [
    'TB0R00010001CS' #Total Inflow M1 : TRUE

    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH

    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService

    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    ]

total_m1_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_df = total_m1_true_df.loc[total_m1_true_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_df['TOTAL_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB0R00010001CS', total_m1_true_df['P'], 0)

total_m1_true_df['PRE_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB1R000900CS', total_m1_true_df['P'], 0)

total_m1_true_df['POST_B2C_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R010500CS', total_m1_true_df['P'], 0)
total_m1_true_df['POST_B2B_T'] = np.where(total_m1_true_df['METRIC_CD']=='TB2R020500CS', total_m1_true_df['P'], 0)

total_m1_true_df['TOL_CON'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000601CS', total_m1_true_df['P'], 0)
total_m1_true_df['TOL_DSV'] = np.where(total_m1_true_df['METRIC_CD']=='TB3R000602CS', total_m1_true_df['P'], 0)

total_m1_true_df['TVS_NOW'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001700CS', total_m1_true_df['P'], 0)
total_m1_true_df['TVS_CMDU'] = np.where(total_m1_true_df['METRIC_CD']=='TB4R001004CS', total_m1_true_df['P'], 0)

total_m1_true_df['CHK_SUM'] = np.where(total_m1_true_df['METRIC_CD'].isin(['TB1R000900CS', 'TB2R010500CS', 'TB2R020500CS', 'TB3R000601CS', 'TB3R000602CS', 'TB4R001700CS', 'TB4R001004CS']), total_m1_true_df['P'], 0)

total_m1_true_df = total_m1_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_df['CHK_DIFF'] = total_m1_true_df['TOTAL_T'] - total_m1_true_df['CHK_SUM']
total_m1_true_df = total_m1_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_df = total_m1_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_df[col] = total_m1_true_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-07-11 04:27:10,0,"319,922,636","319,922,636","225,749,189","61,334,089","4,946,013","25,646,982","829,033","382,335","1,034,995"
1,202502,2025-07-11 04:27:10,0,"288,071,249","288,071,249","192,368,192","63,185,463","4,637,783","25,046,745","1,648,415","267,183","917,468"
2,202503,2025-07-11 04:27:10,0,"310,423,195","310,423,195","201,578,035","70,909,630","5,801,792","28,791,201","2,517,766","470,671","354,099"
3,202504,2025-07-11 04:27:10,0,"295,795,804","295,795,804","191,822,637","69,118,915","5,402,830","26,236,110","2,340,768","441,580","432,964"
4,202505,2025-07-11 04:27:10,0,"287,111,076","287,111,076","190,514,414","59,723,743","6,239,659","28,292,314","1,722,859","401,782","216,305"
5,202506,2025-07-11 04:27:10,0,"264,952,630","264,952,630","173,127,327","56,178,394","6,226,938","27,269,677","1,643,369","330,796","176,129"
6,202507,2025-07-11 04:27:10,-0,"68,681,752","68,681,752","37,521,992","20,177,426","2,012,519","8,546,682","276,608","83,534","62,991"


In [32]:
''' DB0R00010001CS : Total Inflow M1 : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS')

v_metric_list = [
    'DB0R00010001CS' #Total Inflow M1 : DTAC

    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    
    , 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    ]

total_m1_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_df = total_m1_dtac_df.loc[total_m1_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_df['TOTAL_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB0R00010001CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['PRE_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB1R000900CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['POST_B2C_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R010500CS', total_m1_dtac_df['P'], 0)
total_m1_dtac_df['POST_B2B_D'] = np.where(total_m1_dtac_df['METRIC_CD']=='DB2R020500CS', total_m1_dtac_df['P'], 0)

total_m1_dtac_df['CHK_SUM'] = np.where(total_m1_dtac_df['METRIC_CD'].isin(['DB1R000900CS', 'DB2R010500CS', 'DB2R020500CS']), total_m1_dtac_df['P'], 0)

total_m1_dtac_df = total_m1_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_df['CHK_DIFF'] = total_m1_dtac_df['TOTAL_D'] - total_m1_dtac_df['CHK_SUM']
total_m1_dtac_df = total_m1_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_df = total_m1_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_df[col] = total_m1_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-07-11 04:27:10,0,"319,957,964","319,957,964","282,966,592","18,126,426","18,864,945"
1,202502,2025-07-11 04:27:10,0,"287,109,152","287,109,152","249,735,487","17,947,144","19,426,521"
2,202503,2025-07-11 04:27:10,0,"253,094,366","253,094,366","226,924,550","17,637,272","8,532,545"
3,202504,2025-07-11 04:27:10,0,"238,240,343","238,240,343","204,019,741","18,670,677","15,549,925"
4,202505,2025-07-11 04:27:10,0,"245,392,131","245,392,131","209,649,746","19,443,366","16,299,019"
5,202506,2025-07-11 04:27:10,0,"227,331,219","227,331,219","199,804,178","18,544,361","8,982,680"
6,202507,2025-07-11 04:27:10,0,"64,493,570","64,493,570","52,272,319","4,406,313","7,814,938"


### Inflow (Geo Channel)
    B0R00010001CG   Total Inflow M1 - GEO Channel
    TB0R00010001CG  Total Inflow M1 : TRUE - GEO Channel
    DB0R00010001CG  Total Inflow M1 : DTAC - GEO Channel

In [33]:
''' B0R00010001CG : Total Inflow M1 - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'B0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_geo_df = total_m1_geo_df.loc[total_m1_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_geo_df['TOTAL'] = np.where(total_m1_geo_df['METRIC_CD']=='B0R00010001CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['PRE_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['PRE_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['POST_B2C_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2C_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_T'] = np.where(total_m1_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['POST_B2B_D'] = np.where(total_m1_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TOL_CON'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TOL_DSV'] = np.where(total_m1_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['TVS_NOW'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_geo_df['P'], 0)
total_m1_geo_df['TVS_CMDU'] = np.where(total_m1_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_geo_df['P'], 0)

total_m1_geo_df['CHK_SUM'] = np.where(total_m1_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'DB1R000900CG', 'TB2R010500CG', 'DB2R010500CG', 'TB2R020500CG', 'DB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_geo_df['P'], 0)

total_m1_geo_df = total_m1_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_geo_df['CHK_DIFF'] = total_m1_geo_df['TOTAL'] - total_m1_geo_df['CHK_SUM']
total_m1_geo_df = total_m1_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_geo_df = total_m1_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B_T', 'POST_B2B_D', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_geo_df[col] = total_m1_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,POST_B2B_T,POST_B2B_D,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-07-11 04:27:10,0,"491,850,599","491,850,599","223,753,635","206,174,169","31,477,331","7,077,828","2,357,448","381,005","19,471,134","397,930","376,664","383,456"
1,202502,2025-07-11 04:27:10,-0,"438,911,966","438,911,966","190,565,264","184,498,390","32,971,912","7,014,205","2,468,889","382,012","19,961,102","423,582","264,749","361,861"
2,202503,2025-07-11 04:27:10,0,"448,075,830","448,075,830","199,692,427","174,812,829","38,509,836","7,839,038","3,206,656","392,224","22,964,967","133,660","470,449","53,744"
3,202504,2025-07-11 04:27:10,-0,"429,413,620","429,413,620","189,643,908","169,763,448","37,483,652","7,699,989","2,886,311","387,725","20,901,734","134,885","441,246","70,723"
4,202505,2025-07-11 04:27:10,0,"430,885,163","430,885,163","188,609,826","178,781,371","29,453,492","7,516,555","3,110,301","399,840","22,391,575","221,363","400,240",600
5,202506,2025-07-11 04:27:10,0,"396,063,748","396,063,748","171,326,357","166,446,894","26,181,899","6,944,013","2,765,249","485,883","21,482,408","94,361","330,703","5,981"
6,202507,2025-07-11 04:27:10,0,"99,380,114","99,380,114","37,173,908","42,714,449","9,397,611","2,215,791","821,467","139,574","6,785,018","42,781","83,534","5,981"


In [34]:
''' TB0R00010001CG : Total Inflow M1 : TRUE - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG')

v_metric_list = [
    'TB0R00010001CG' #Total Inflow M1 - GEO Channel
    
    , 'TB1R000900CG' #Prepaid Inflow M1 : TMH - GEO Channel
    
    , 'TB2R010500CG' #Postpaid Inflow M1 B2C : TMH - GEO Channel
    , 'TB2R020500CG' #Postpaid Inflow M1 B2B : TMH - GEO Channel
    
    , 'TB3R000601CG' #TOL Inflow M1 Connected : Consumer - GEO Channel
    , 'TB3R000602CG' #TOL Inflow M1 Connected : DataService - GEO Channel
    
    , 'TB4R001700CG' #TVS Now Inflow M1 - GEO Channel
    , 'TB4R001004CG' #TVS CMDU Inflow M1 - GEO Channel
    ]

total_m1_true_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_true_geo_df = total_m1_true_geo_df.loc[total_m1_true_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_true_geo_df['TOTAL_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB0R00010001CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['PRE_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB1R000900CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['POST_B2C_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R010500CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['POST_B2B_T'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB2R020500CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TOL_CON'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000601CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TOL_DSV'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB3R000602CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['TVS_NOW'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001700CG', total_m1_true_geo_df['P'], 0)
total_m1_true_geo_df['TVS_CMDU'] = np.where(total_m1_true_geo_df['METRIC_CD']=='TB4R001004CG', total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df['CHK_SUM'] = np.where(total_m1_true_geo_df['METRIC_CD'].isin(['TB1R000900CG', 'TB2R010500CG', 'TB2R020500CG', 'TB3R000601CG', 'TB3R000602CG', 'TB4R001700CG', 'TB4R001004CG']), total_m1_true_geo_df['P'], 0)

total_m1_true_geo_df = total_m1_true_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'POST_B2B_T':'sum', 'TOL_CON':'sum', 'TOL_DSV':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum', 'CHK_SUM':'sum'})
total_m1_true_geo_df['CHK_DIFF'] = total_m1_true_geo_df['TOTAL_T'] - total_m1_true_geo_df['CHK_SUM']
total_m1_true_geo_df = total_m1_true_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_true_geo_df = total_m1_true_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'POST_B2B_T', 'TOL_CON', 'TOL_DSV', 'TVS_NOW', 'TVS_CMDU']]

mod_col_list = total_m1_true_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_true_geo_df[col] = total_m1_true_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_true_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,POST_B2B_T,TOL_CON,TOL_DSV,TVS_NOW,TVS_CMDU
0,202501,2025-07-11 04:27:10,0,"278,217,598","278,217,598","223,753,635","31,477,331","2,357,448","19,471,134","397,930","376,664","383,456"
1,202502,2025-07-11 04:27:10,0,"247,017,359","247,017,359","190,565,264","32,971,912","2,468,889","19,961,102","423,582","264,749","361,861"
2,202503,2025-07-11 04:27:10,0,"265,031,740","265,031,740","199,692,427","38,509,836","3,206,656","22,964,967","133,660","470,449","53,744"
3,202504,2025-07-11 04:27:10,-0,"251,562,458","251,562,458","189,643,908","37,483,652","2,886,311","20,901,734","134,885","441,246","70,723"
4,202505,2025-07-11 04:27:10,0,"244,187,397","244,187,397","188,609,826","29,453,492","3,110,301","22,391,575","221,363","400,240",600
5,202506,2025-07-11 04:27:10,0,"222,186,959","222,186,959","171,326,357","26,181,899","2,765,249","21,482,408","94,361","330,703","5,981"
6,202507,2025-07-11 04:27:10,0,"54,310,300","54,310,300","37,173,908","9,397,611","821,467","6,785,018","42,781","83,534","5,981"


In [35]:
''' DB0R00010001CG : Total Inflow M1 : DTAC - GEO Channel '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG')

v_metric_list = [
    'DB0R00010001CG' #Total Inflow M1 : DTAC - GEO Channel
    
    , 'DB1R000900CG' #Prepaid Inflow M1 : DTAC - GEO Channel
    
    , 'DB2R010500CG' #Postpaid Inflow M1 B2C : DTAC - GEO Channel
    , 'DB2R020500CG' #Postpaid Inflow M1 B2B : DTAC - GEO Channel
    ]

total_m1_dtac_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_m1_dtac_geo_df = total_m1_dtac_geo_df.loc[total_m1_dtac_geo_df['METRIC_CD'].isin(v_metric_list)]

total_m1_dtac_geo_df['TOTAL_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB0R00010001CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['PRE_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB1R000900CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['POST_B2C_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R010500CG', total_m1_dtac_geo_df['P'], 0)
total_m1_dtac_geo_df['POST_B2B_D'] = np.where(total_m1_dtac_geo_df['METRIC_CD']=='DB2R020500CG', total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df['CHK_SUM'] = np.where(total_m1_dtac_geo_df['METRIC_CD'].isin(['DB1R000900CG', 'DB2R010500CG', 'DB2R020500CG']), total_m1_dtac_geo_df['P'], 0)

total_m1_dtac_geo_df = total_m1_dtac_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'POST_B2B_D':'sum', 'CHK_SUM':'sum'})
total_m1_dtac_geo_df['CHK_DIFF'] = total_m1_dtac_geo_df['TOTAL_D'] - total_m1_dtac_geo_df['CHK_SUM']
total_m1_dtac_geo_df = total_m1_dtac_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_m1_dtac_geo_df = total_m1_dtac_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D', 'POST_B2B_D']]

mod_col_list = total_m1_dtac_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_m1_dtac_geo_df[col] = total_m1_dtac_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_m1_dtac_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D,POST_B2B_D
0,202501,2025-07-11 04:27:10,-0,"213,633,001","213,633,001","206,174,169","7,077,828","381,005"
1,202502,2025-07-11 04:27:10,0,"191,894,607","191,894,607","184,498,390","7,014,205","382,012"
2,202503,2025-07-11 04:27:10,0,"183,044,091","183,044,091","174,812,829","7,839,038","392,224"
3,202504,2025-07-11 04:27:10,0,"177,851,162","177,851,162","169,763,448","7,699,989","387,725"
4,202505,2025-07-11 04:27:10,0,"186,697,766","186,697,766","178,781,371","7,516,555","399,840"
5,202506,2025-07-11 04:27:10,0,"173,876,790","173,876,790","166,446,894","6,944,013","485,883"
6,202507,2025-07-11 04:27:10,0,"45,069,814","45,069,814","42,714,449","2,215,791","139,574"
